# Generación de Contenido de Marketing con IA para ESU Analytics  

Este cuaderno de Jupyter presenta una **prueba de concepto (POC)** orientada a la generación de contenido de marketing digital para **ESU Analytics**, una consultora especializada en acompañar a PyMEs de los sectores manufacturero y comercial en la profesionalización de su estrategia de datos.  

La POC tiene como propósito:  
- **Demostrar** cómo aplicar técnicas de *Fast Prompting* para optimizar la creación de publicaciones en LinkedIn.  
- **Experimentar** con diferentes configuraciones de prompts para evaluar su impacto en la claridad, relevancia y consistencia del contenido generado.  
- **Controlar costos y eficiencia**, implementando límites de tokens y buenas prácticas de uso del modelo `gpt-4o-mini` de OpenAI.  
- **Garantizar trazabilidad**, exportando cada ejecución con su *input* (brief y prompt generado) y *output* (texto final y métricas de tokens).  

De esta manera, se busca comprobar si las técnicas aprendidas permiten mejorar la propuesta de solución planteada en la etapa anterior, asegurando un flujo de generación de contenido **más ágil, profesional y reproducible**, que apoye la estrategia de visibilidad digital de ESU Analytics.  


## Parámetros del brief

In [1]:
# Parámetros del brief que guían el prompt

# Ejemplo comentado de valores posibles:
# brief_tema = "Cómo ordenar datos de ventas para tomar decisiones mejor informadas",
brief_tema = "Cómo ordenar datos de ventas para tomar decisiones mejor informadas",
# brief_objetivo = "Educar y generar interés por un diagnóstico inicial",
brief_objetivo = "Educar y generar interés por un diagnóstico inicial",
# brief_audiencia = "Dueños de PyMEs retail y gerentes comerciales",
brief_audiencia = "Dueños de PyMEs retail y gerentes comerciales",
# brief_insight = "Sin datos limpios y consistentes, el margen se vuelve impredecible",
brief_insight = "Sin datos limpios y consistentes, el margen se vuelve impredecible",
# brief_evidencia = "Planillas dispersas generan errores; unificarlas reduce reprocesos",
brief_evidencia = "Planillas dispersas generan errores; unificarlas reduce reprocesos",
# brief_cta = "Agendá una consulta gratuita para revisar tu caso",
brief_cta = "Agendá una consulta gratuita para revisar tu caso",
# brief_restricciones = "máx. 110 palabras, sin hashtags ni emojis"
brief_restricciones = "máx. 110 palabras, sin hashtags ni emojis"

## 📦 0. Setup (variables y helpers)

In [2]:
# Importar librerías y configurar cliente
import os, time, json
from dotenv import load_dotenv
import tiktoken
from openai import OpenAI

In [3]:
# Cargar la API Key desde .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Tokenizer para modelos GPT-4o/mini
ENCODING_NAME = "o200k_base"
enc = tiktoken.get_encoding(ENCODING_NAME)

In [4]:
# Funciones auxiliares
def truncate_by_tokens(text: str, max_tokens: int) -> str:
    tokens = enc.encode(text)
    if len(tokens) <= max_tokens:
        return text
    return enc.decode(tokens[:max_tokens])

def count_tokens(text: str) -> int:
    return len(enc.encode(text))

def now_ms():
    return round(time.time() * 1000)

## 🧪 1. Prompt builder (variables)

In [5]:
SYSTEM_PROMPT = """Eres un creador de contenido para ESU Analytics, consultora que ayuda a PyMEs de Manufactura y Comercio a profesionalizar su estrategia de datos (ventas, stock, estrategia comercial).
Tu prioridad: claridad, utilidad práctica para dueños/gerentes PyME en Argentina, tono profesional y cercano, y una llamada a la acción concreta.
Responde siempre en español de Buenos Aires neutral.
"""

USER_TEMPLATE = """Tarea: Escribe un post de LinkedIn.

Brief:
- Tema: {tema}
- Objetivo del post: {objetivo}
- Audiencia: {audiencia}
- Ángulo/Insight: {insight}
- Evidencia/ejemplo breve (opcional): {evidencia}
- CTA: {cta}
- Restricciones: {restricciones}

Formato deseado:
- 3–5 líneas, con una idea central y un cierre con CTA.
- Incluye 1 micro-lista de 3 ítems si aporta claridad.
- No uses hashtags en esta versión.

Entrega:
Devuelve SOLO un objeto JSON con:
{{
 "titulo": "...",
 "cuerpo": "...",
 "cta": "...",
 "palabras": <int>
}}
"""

def build_user_prompt(tema, objetivo, audiencia, insight, evidencia, cta, restricciones):
    return USER_TEMPLATE.format(
        tema=tema,
        objetivo=objetivo,
        audiencia=audiencia,
        insight=insight,
        evidencia=evidencia or "—",
        cta=cta,
        restricciones=restricciones
    )

## ⚙️ 2. Llamada a OpenAI con límites de costo

In [6]:
def generate_post(brief, 
                  max_input_tokens=800, 
                  max_output_tokens=220, 
                  temperature=0.3):
    user_prompt = build_user_prompt(**brief)
    user_prompt = truncate_by_tokens(user_prompt, max_input_tokens)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_prompt}
    ]

    t0 = now_ms()
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=max_output_tokens,
        temperature=temperature
    )
    dt = now_ms() - t0

    text = resp.choices[0].message.content

    try:
        data = json.loads(text)
    except Exception:
        data = {"raw": text}

    meta = {
        "elapsed_ms": dt,
        "input_tokens_est": count_tokens(SYSTEM_PROMPT) + count_tokens(user_prompt),
        "output_tokens_est": count_tokens(text),
        "total_tokens_est": count_tokens(SYSTEM_PROMPT) + count_tokens(user_prompt) + count_tokens(text),
        "api": "chat.completions",
        "model": "gpt-4o-mini",
        "max_output_tokens": max_output_tokens,
        "temperature": temperature
    }
    # 👇 devolvemos también el prompt de usuario final para trazabilidad
    return data, meta, user_prompt

In [ ]:
# Helpers de exportación
import re
from datetime import datetime

def ensure_export_dir(path="Exportado"):
    os.makedirs(path, exist_ok=True)
    return path

def next_post_id(path="Exportado"):
    """
    Escanea 'Exportado' y devuelve el próximo ID tipo '001', '002', ...
    Detecta archivos 'postXYZ.json' o 'postXYZ.md'
    """
    if not os.path.isdir(path):
        return "001"
    max_n = 0
    for fname in os.listdir(path):
        m = re.match(r"post(\d{3})\.(json|md)$", fname, re.IGNORECASE)
        if m:
            n = int(m.group(1))
            if n > max_n:
                max_n = n
    return f"{max_n+1:03d}"

def export_run(data, meta, brief, user_prompt, export_dir="Exportado"):
    """
    Guarda:
      - postXYZ.json con input, prompts, output y meta
      - postXYZ.md con texto listo para publicar
    """
    ensure_export_dir(export_dir)
    run_id = next_post_id(export_dir)
    ts = datetime.now().isoformat(timespec="seconds")

    # Paquete JSON completo (para trazabilidad)
    payload = {
        "timestamp": ts,
        "project": "ESU Analytics - Marketing con IA",
        "inputs": {
            "brief": brief,
            "system_prompt": SYSTEM_PROMPT,
            "user_prompt": user_prompt
        },
        "output": data,
        "meta": meta
    }

    json_path = os.path.join(export_dir, f"post{run_id}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

    # Markdown “listo para publicar”
    md_lines = []
    md_lines.append(f"# {data.get('titulo','(Sin título)')}")
    md_lines.append("")
    cuerpo = data.get("cuerpo") or data.get("raw") or ""
    md_lines.append(cuerpo)
    md_lines.append("")
    if "cta" in data and data["cta"]:
        md_lines.append(f"**CTA:** {data['cta']}")
        md_lines.append("")
    if "palabras" in data:
        md_lines.append(f"_Palabras: {data['palabras']}_")
        md_lines.append("")
    md_lines.append(f"<!-- post ID: {run_id} | {ts} | modelo: {meta.get('model')} | tokens aprox: {meta.get('total_tokens_est')} -->")

    md_path = os.path.join(export_dir, f"post{run_id}.md")
    with open(md_path, "w", encoding="utf-8") as f:
        f.write("\n".join(md_lines))

    return {"json": json_path, "md": md_path, "id": run_id}


## 🧪 3. Ejecución A/B (experimentos rápidos)

In [8]:
brief = {
    "tema": brief_tema,
    "objetivo": brief_objetivo,
    "audiencia": brief_audiencia,
    "insight": brief_insight,
    "evidencia": brief_evidencia,
    "cta": brief_cta,
    "restricciones": brief_restricciones
}

## 📊 4. Micro-evaluación (manual / semiautomática)

In [9]:
post, meta, user_prompt = generate_post(brief, temperature=0.2, max_output_tokens=220)
paths = export_run(post, meta, brief, user_prompt, export_dir="Exportado")

print("\n===== POST GENERADO =====")
print(json.dumps(post, ensure_ascii=False, indent=2))
print("\n===== MÉTRICAS =====")
print(meta)
print("\n===== ARCHIVOS GUARDADOS =====")
print(paths)


===== POST GENERADO =====
{
  "titulo": "Cómo ordenar datos de ventas para decisiones más informadas",
  "cuerpo": "Sin datos limpios y consistentes, el margen de tu negocio se vuelve impredecible. Para evitar errores y mejorar tu estrategia comercial, considera estos pasos: \n1. Unifica tus planillas de ventas. \n2. Establece un formato estándar. \n3. Realiza auditorías periódicas. \nOrdenar tus datos no solo reduce reprocesos, sino que también te permite tomar decisiones más acertadas.",
  "cta": "Agendá una consulta gratuita para revisar tu caso.",
  "palabras": 102
}

===== MÉTRICAS =====
{'elapsed_ms': 3390, 'input_tokens_est': 312, 'output_tokens_est': 138, 'total_tokens_est': 450, 'api': 'chat.completions', 'model': 'gpt-4o-mini', 'max_output_tokens': 220, 'temperature': 0.2}

===== ARCHIVOS GUARDADOS =====
{'json': 'Exportado\\post001.json', 'md': 'Exportado\\post001.md', 'id': '001'}
